<a href="https://colab.research.google.com/github/anushkaa-10/TechTwish_amazon/blob/main/TechTwis_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from pathlib import Path
from PIL import Image, ImageEnhance
from tqdm import tqdm
import multiprocessing

dataset_path = "/content/drive/My Drive/resources/dataset/train_with_images.csv"
preprocessed_folder = "/content/drive/My Drive/resources/preprocessed_images/"

max_images = 1000

In [ ]:

df = pd.read_csv(dataset_path)

df = df.head(max_images)

In [ ]:

resize_dim = (128, 128)
augmentation_factor = 5
num_workers = multiprocessing.cpu_count()

# Preprocessing function
def preprocess_image(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return None

        # Normalize the image
        gray_img_normalized = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

        # Resize the image
        resized_img = cv2.resize(gray_img_normalized, resize_dim, interpolation=cv2.INTER_AREA)
        return resized_img
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Function to augment and save the image
def augment_and_save(image, base_output_path):
    augmented_images = []
    for i in range(augmentation_factor):
        angle = np.random.uniform(-30, 30)
        M = cv2.getRotationMatrix2D((resize_dim[0] / 2, resize_dim[1] / 2), angle, 1)
        rotated_img = cv2.warpAffine(image, M, resize_dim, flags=cv2.INTER_LINEAR)

        # Random flipping
        flipped_img = cv2.flip(rotated_img, np.random.choice([-1, 0, 1]))

        # Brightness adjustment
        pil_img = Image.fromarray(flipped_img)
        enhancer = ImageEnhance.Brightness(pil_img)
        brightness_factor = np.random.uniform(0.5, 1.5)
        bright_img = enhancer.enhance(brightness_factor)
        augmented_images.append(np.array(bright_img))
    return augmented_images

In [ ]:
def process_images(df, batch_size=1000):
    batch = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        image_path = row['image_path']
        output_path = os.path.join(preprocessed_folder, Path(image_path).name)

        # Preprocess image
        processed_image = preprocess_image(output_path)
        if processed_image is not None:
            batch.append(processed_image)

            # Augment the image
            augmented_images = augment_and_save(processed_image, output_path)
            batch.extend(augmented_images)

        # Save in batches for memory efficiency
        if len(batch) >= batch_size:
            yield batch
            batch = []

    if batch:
        yield batch

In [ ]:
def parallel_processing(df, num_workers=num_workers, batch_size=1000):
    with multiprocessing.Pool(num_workers) as pool:
        for processed_batch in process_images(df, batch_size=batch_size):
            print(f"Processed batch of {len(processed_batch)} images")


In [ ]:

parallel_processing(df)

print("Processing completed for 10,000 images.")


 17%|█▋        | 167/1000 [02:13<10:04,  1.38it/s]

Processed batch of 1002 images


 33%|███▎      | 334/1000 [04:17<08:33,  1.30it/s]

Processed batch of 1002 images


 50%|█████     | 501/1000 [06:22<05:55,  1.40it/s]

...

Processed batch of 1002 images


 84%|████████▎ | 835/1000 [10:34<02:18,  1.19it/s]

Processed batch of 1002 images


100%|██████████| 1000/1000 [11:08<00:00,  1.50it/s]

Processed batch of 270 images
Processing completed for 10,000 images.


In [ ]:
import os
from PIL import Image

preprocessed_folder = "/content/drive/My Drive/resources/preprocessed_images/"

def verify_images(folder_path):
    files = os.listdir(folder_path)
    num_files = len(files)
    print(f"Number of images in '{folder_path}': {num_files}")

    for file in files[:5]:
        try:
            img_path = os.path.join(folder_path, file)
            img = Image.open(img_path)
            print(f"Image '{file}' opened successfully.")
        except Exception as e:
            print(f"Error opening image '{file}': {e}")

verify_images(preprocessed_folder)

Number of images in '/content/drive/My Drive/resources/preprocessed_images/': 5216
Image '71aSE8GfbQL_aug_3.jpg' opened successfully.
Image '71aSE8GfbQL_aug_4.jpg' opened successfully.
Image '71vCZl9YsoL.jpg' opened successfully.
Image '71vCZl9YsoL_aug_0.jpg' opened successfully.
Image '71vCZl9YsoL_aug_1.jpg' opened successfully.


In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,885 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import pytesseract
from PIL import Image
import pandas as pd
import os

def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

preprocessed_folder = "/content/drive/My Drive/resources/preprocessed_images/"

image_files = [os.path.join(preprocessed_folder, file) for file in os.listdir(preprocessed_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

ocr_results = []

for image_path in image_files:
    text = extract_text_from_image(image_path)
    ocr_results.append({'image_path': image_path, 'ocr_text': text})

df_ocr = pd.DataFrame(ocr_results)
df_ocr.to_csv("/content/drive/My Drive/resources/dataset/train_with_ocr.csv", index=False)

print("OCR processing completed and results saved.")


OCR processing completed and results saved.


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

ocr_data = pd.read_csv("/content/drive/My Drive/resources/dataset/train_with_ocr.csv")

IMG_SIZE = (128, 128)

def load_and_preprocess_images(image_paths, img_size=IMG_SIZE):
    images = []
    for path in image_paths:
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, img_size)
            img = img / 255.0
            images.append(img)
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            images.append(np.zeros(img_size + (3,)))
    return np.array(images)

image_paths = ocr_data['image_path'].values
images = load_and_preprocess_images(image_paths)

print(ocr_data.columns)

labels = ocr_data['ocr_text'].values

le = LabelEncoder()
labels = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

batch_size = 32
epochs = 10

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs
)

model.save("/content/drive/My Drive/resources/models/cnn_model.h5")

print("Model training completed and model saved.")

Index(['image_path', 'ocr_text'], dtype='object')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


130/130 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.7962 - loss: 2.8167 - val_accuracy: 0.8094 - val_loss: 1.8911
Epoch 2/10
  1/130 ━━━━━━━━━━━━━━━━━━━━ 1:41 784ms/step - accuracy: 0.8750 - loss: 1.6801

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


130/130 ━━━━━━━━━━━━━━━━━━━━ 14s 106ms/step - accuracy: 0.8750 - loss: 1.6801 - val_accuracy: 0.8094 - val_loss: 1.9196
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.8252 - loss: 1.8274 - val_accuracy: 0.8094 - val_loss: 2.1288
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - accuracy: 0.8438 - loss: 1.5969 - val_accuracy: 0.8094 - val_loss: 2.1655
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.8330 - loss: 1.6932 - val_accuracy: 0.8094 - val_loss: 2.0714
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 11s 79ms/step - accuracy: 0.7812 - loss: 2.1554 - val_accuracy: 0.8094 - val_loss: 2.0635
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.8252 - loss: 1.7716 - val_accuracy: 0.8094 - val_loss: 2.1890
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - accuracy: 0.8750 - loss: 1.3013 - val_accuracy: 0.8094 - val_loss: 2.2100
Epoch 9/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.8279 - loss: 1.7317 - val_accuracy: 0

Model training completed and model saved.


In [ ]:
from sklearn.metrics import f1_score, classification_report


y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"F1 Score: {f1:.4f}")

report = classification_report(y_test, y_pred, target_names=[str(c) for c in le.classes_])
print("Classification Report:")
print(report)

33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step
F1 Score: 0.7241
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         1
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         1
          47       0.00      0.00      0.00         1
          56       0.00      0.00      0.00         1
          62       0.00      0.00      0.00         1
          78       0.00      0.00      0.00         2
          80       0.00      0.00      0.00         1
          86       0.00      0.00      0.00         1
          92       0.00      0.00      0.00         1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8094


In [ ]:
from sklearn.metrics import accuracy_score

# Step 1: Make predictions on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)

# Step 2: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy:.4f}")


33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 292ms/step
Accuracy: 0.8094


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/resources/student_resource 3/src/')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (1000, 1000, 3)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/resources/student_resource 3/src/')



In [ ]:

import pandas as pd
from utils import download_images

test_data = pd.read_csv('/content/drive/MyDrive/resources/student_resource 3/dataset/test.csv')

download_images(test_data['image_link'], '/path/to/store/images')


100%|██████████| 131187/131187 [03:33<00:00, 614.71it/s]


In [ ]:
print(test_data.columns)

Index(['index', 'image_link', 'group_id', 'entity_name'], dtype='object')


In [ ]:
import numpy as np
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import cv2
from tensorflow.keras.models import load_model

def download_image(image_url, max_size=500):
    try:
        response = requests.get(image_url)
        response.raise_for_status()

        img = Image.open(BytesIO(response.content))

        img.thumbnail((max_size, max_size), Image.ANTIALIAS)

        img = np.array(img)
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        if img.shape[-1] != 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        return img
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None

def predict_images(test_data, model, target_size):
    predictions = []
    for idx, row in test_data.iterrows():
        img_url = row['image_link']

        print(f"Loading image from URL: {img_url}")

        img = download_image(img_url)

        if img is not None:
            print(f"Original image shape: {img.shape}")

            img = cv2.resize(img, target_size)
            img = img.astype('float32') / 255.0

            print(f"Resized image shape: {img.shape}")

            img = np.expand_dims(img, axis=0)
            pred = model.predict(img)

            print(f"Raw prediction: {pred}")

            predicted_class = np.argmax(pred, axis=-1)

            prediction_str = f"{predicted_class} unit"

            predictions.append(prediction_str)
        else:
            print(f"Error: Unable to process image from URL {img_url}")
            predictions.append("")
    return predictions

def save_predictions(test_data, predictions, output_file):
    test_data['prediction'] = predictions
    test_data[['index', 'prediction']].to_csv(output_file, index=False)

model = load_model('/content/drive/MyDrive/resources/models/cnn_model.h5')

model.summary()

target_size = (126, 126)

test_data = pd.read_csv('/content/drive/MyDrive/resources/student_resource 3/dataset/test.csv')

predictions = predict_images(test_data, model, target_size)

output_file = '/content/drive/MyDrive/resources/student_resource 3/output/test_out.csv'
save_predictions(test_data, predictions, output_file)

print(f"Predictions saved to {output_file}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 837)                 │         107,973 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,500,295 (28.61 MB)

 Trainable params: 7,500,293 (28.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Loading image from URL: https://m.media-amazon.com/images/I/110EibNyclL.jpg
Original image shape: (500, 500, 3)
Resized image shape: (126, 126, 3)


<ipython-input-29-9cb4903d9669>:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail((max_size, max_size), Image.ANTIALIAS)


Streaming output truncated to the last 5000 lines.
  2.54949173e-10 3.46928930e-10 7.97482747e-10 1.82273346e-11
  1.24829356e-07 4.57332588e-10 2.02527523e-10 3.17411014e-10
  4.96994557e-10 2.22529165e-11 1.25941873e-13 2.50302834e-10
  6.00372807e-11 2.37156184e-09 3.82695542e-10 1.16414134e-09
  1.04382814e-09 9.73341091e-13 3.40149964e-10 1.35490313e-10
  3.39632272e-12 3.06813930e-09 3.94798416e-10 1.34301625e-10
  1.42584664e-12 1.90675964e-09 1.10737686e-09 2.27134797e-10
  8.86127005e-10 1.54411239e-08 2.48143872e-13 2.15352824e-09
  6.00193450e-10 3.48203932e-09 1.22206600e-09 1.28814889e-10
  9.90233340e-10 2.54384380e-09 1.22400617e-10 2.42317993e-13
  5.04259023e-10 5.77537530e-13 1.22541373e-08 4.99887465e-10
  1.90340588e-09 3.79264981e-10 2.44809988e-12 6.72503997e-10
  1.75483031e-10 9.62711466e-10 2.12375880e-13 1.17302779e-09
  1.05520825e-09 2.28238478e-12 7.87801047e-10 1.33062505e-09
  1.24788413e-09 1.67638229e-13 1.00121564e-10 7.21959914e-09
  7.07264081e-10 2.

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Function to preprocess images
# Function to preprocess images
def preprocess_image(image_path, target_size=(120, 120)): # Change target_size to 120x120
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
from keras.models import load_model

# Load your pre-trained model
model = load_model('/content/drive/MyDrive/resources/models/cnn_model.h5')


In [ ]:
# Directory where images were stored after downloading
image_dir = '/path/to/store/images'

# Function to process images and predict
def predict_images(test_data, image_dir, model):
    predictions = []
    for index, row in test_data.iterrows():
        image_path = os.path.join(image_dir, os.path.basename(row['image_link']))

        # Preprocess the image
        img_array = preprocess_image(image_path)

        # Predict using the model
        prediction = model.predict(img_array)

        # Assuming prediction gives a numeric value and a unit needs to be appended
        prediction_value = prediction[0][0]
        prediction_str = f"{prediction_value:.2f} gram"  # Adjust unit accordingly

        predictions.append(prediction_str)
    return predictions


In [ ]:
from PIL import Image
import numpy as np

def preprocess_image(image_path):
    img = Image.open(image_path).resize((image_size, image_size))  # Resize image as per your model input size
    img = np.array(img) / 255.0  # Normalize if needed
    return np.expand_dims(img, axis=0)  # Add batch dimension


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/drive/MyDrive/resources/models/cnn_model.h5')
